# Shamelessly inspired by https://www.kaggle.com/code/yuanzhezhou/jane-street-baseline-lgb-xgb-and-catboost/notebook

In [ ]:
!pip install --quiet -r requirements.txt

In [ ]:
!nvidia-smi

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os, gc
import pandas as pd
import polars as pl

import lightgbm as lgb
import xgboost as xgb
#import catboost as cbt # needs numpy <2.0
import joblib
from joblib import Parallel, delayed
import numpy as np
#from hyperopt import hp, fmin, tpe, Trials
#from hyperopt.pyll.base import scope
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
#from tqdm.notebook import tqdm
#from joblib import dump, load
#import datatable as dtable
#from mlxtend.evaluate import GroupTimeSeriesSplit
import kaggle_evaluation.jane_street_inference_server as js_server

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
from torchvision import datasets
from torchvision.transforms import ToTensor
TEST=True


In [ ]:
# check that torch is working and sees the GPU
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

In [ ]:
collected = gc.collect()
# Prints Garbage collector 
# as 0 object
print("Garbage collector: collected",
          "%d objects." % collected)

# Load data

In [ ]:
path = os.getcwd()
if TEST:
    df1 = pl.read_parquet(path+'/input/train.parquet/partition_id=7/part-0.parquet')
    df2 = pl.read_parquet(path+'/input/train.parquet/partition_id=8/part-0.parquet')
    df3 = pl.read_parquet(path+'/input/train.parquet/partition_id=9/part-0.parquet')
    train_df = pl.concat([df1,df2, df3])
    del df1, df2, df3
else:
    # very important to optimize memory when loading the full damn thing
    cols = ['date_id', 'time_id', 'weight', 'symbol_id']
    cols.extend(['feature_{:02d}'.format(num) for num in range(0, 79)])
    cols.append('responder_6')
    df1 = pl.read_parquet(path+'/input/train.parquet/partition_id=0/part-0.parquet', columns=cols)
    df2 = pl.read_parquet(path+'/input/train.parquet/partition_id=1/part-0.parquet', columns=cols)
    df3 = pl.read_parquet(path+'/input/train.parquet/partition_id=2/part-0.parquet', columns=cols)
    df4 = pl.read_parquet(path+'/input/train.parquet/partition_id=3/part-0.parquet', columns=cols)
    df5 = pl.read_parquet(path+'/input/train.parquet/partition_id=4/part-0.parquet', columns=cols)
    df6 = pl.read_parquet(path+'/input/train.parquet/partition_id=5/part-0.parquet', columns=cols)
    df7 = pl.read_parquet(path+'/input/train.parquet/partition_id=6/part-0.parquet', columns=cols)
    df8 = pl.read_parquet(path+'/input/train.parquet/partition_id=7/part-0.parquet', columns=cols)
    df9 = pl.read_parquet(path+'/input/train.parquet/partition_id=8/part-0.parquet', columns=cols)
    df10 = pl.read_parquet(path+'/input/train.parquet/partition_id=9/part-0.parquet', columns=cols)
    train_df = pl.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10])
    del df1, df2, df3, df4, df5, df6, df7, df8, df9, df10

lags = pl.read_parquet(path+'/input/lags.parquet/date_id=0/part-0.parquet')
feature_tags = pl.read_csv(path+'/input/features.csv') # no one seems to use this....
gc.collect()

In [ ]:
features = [c for c in train_df.columns if 'feature' in c]
#features.append('weight')
#features.append('time_id')
#features.append('symbol_id')
target = 'responder_6'
print(train_df.shape)
print(train_df.columns)
print(train_df[features].null_count())
# Returns the number of
# objects it has collected
# and deallocated
collected = gc.collect()
# Prints Garbage collector 
# as 0 object
print("Garbage collector: collected",
          "%d objects." % collected)

# note to self, incorporate features.csv true/false tables at some point 

# Preprocessing

In [ ]:
print('Filling...')
# next two lines were for pandas not polars
#f_mean = train[features].mean() 
# print(train.weight.gt(0).sum() == train.shape[0]) # check if weights>0 is true for entire dataframe
print(train_df.shape)
train_df = train_df.fill_null(0)
#train.dropna(inplace=True)
print(train_df.null_count())

In [ ]:
# for the purged group time series split, code is copied from somewhere
# TODO: make GitHub GIST
# TODO: add as dataset
# TODO: add logging with verbose

from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.|
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
# from matplotlib.colors import ListedColormap
    
# # this is code slightly modified from the sklearn docs here:
# # https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py
# def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=10):
#     """Create a sample plot for indices of a cross-validation object."""
    
#     cmap_cv = plt.cm.coolwarm

#     jet = plt.cm.get_cmap('jet', 256)
#     seq = np.linspace(0, 1, 256)
#     _ = np.random.shuffle(seq)   # inplace
#     cmap_data = ListedColormap(jet(seq))

#     # Generate the training/testing visualizations for each CV split
#     for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
#         # Fill in indices with the training/test groups
#         indices = np.array([np.nan] * len(X))
#         indices[tt] = 1
#         indices[tr] = 0

#         # Visualize the results
#         ax.scatter(range(len(indices)), [ii + .5] * len(indices),
#                    c=indices, marker='_', lw=lw, cmap=cmap_cv,
#                    vmin=-.2, vmax=1.2)

#     # Plot the data classes and groups at the end
#     ax.scatter(range(len(X)), [ii + 1.5] * len(X),
#                c=y, marker='_', lw=lw, cmap=plt.cm.Set3)

#     ax.scatter(range(len(X)), [ii + 2.5] * len(X),
#                c=group, marker='_', lw=lw, cmap=cmap_data)

#     # Formatting
#     yticklabels = list(range(n_splits)) + ['target', 'day']
#     ax.set(yticks=np.arange(n_splits+2) + .5, yticklabels=yticklabels,
#            xlabel='Sample index', ylabel="CV iteration",
#            ylim=[n_splits+2.2, -.2], xlim=[0, len(y)])
#     ax.set_title('{}'.format(type(cv).__name__), fontsize=15)
#     return ax


# fig, ax = plt.subplots()
# #plot_cv_indices(cv, X_train, y_train, groups, ax, 5, lw=20)
# plot_cv_indices(
#     cv,
#     train[features].values,
#     train['responder_6'].values,
#     train['date_id'].values,
#     ax,
#     5,
#     lw=20
# )
# rubbish=gc.collect()

In [ ]:
TRAINING=True
num_valid_dates = 100
skip_dates = 0
N_fold = 5

# filter df to skip specified num of dates
train_df = train_df.filter(pl.col('date_id') > skip_dates)

# get unique dates as np list
dates = train_df['date_id'].unique().to_numpy()

# specify train and validation dates
valid_dates = dates[-num_valid_dates:]
train_dates = dates[:-num_valid_dates]

# split train_dates into N_fold sub-arrays (not necessarily equal size)
train_dates = np.array_split(train_dates, N_fold)
#print(train_dates)

In [ ]:
#extract needed parameters from dataframe as np arrays, will later turn into tensors
X_vl = train_df.filter(pl.col('date_id') >= valid_dates.min())[features].to_numpy()
y_vl = train_df.filter(pl.col('date_id') >= valid_dates.min())['responder_6'].to_numpy()
w_vl = train_df.filter(pl.col('date_id') >= valid_dates.min())['weight'].to_numpy()

# neat hack to ensure the original dataframe is gone
# del train
# gc.collect()
# train = pd.DataFrame()

In [ ]:
print(X_vl.shape)
print(w_vl.shape)

In [ ]:
# Initialize list to store trained models
models = []

model_path = './models'
# Function to train a model or load a pre-trained model
def train(model_dict, model_name='lgb'):

    if TRAINING:
        # Select dates for training based on fold number
        selected_dates = train_dates[i] # i exists outside this function but in the cell
    
        # Get the model from the dictionary
        model = model_dict[model_name]
    
        # Extract features, target, weights for selected training dates
        X_tr = train_df.filter(pl.col('date_id').is_between(selected_dates[0], selected_dates[-1]))[features].to_numpy()
        y_tr = train_df.filter(pl.col('date_id').is_between(selected_dates[0], selected_dates[-1]))['responder_6'].to_numpy()
        w_tr = train_df.filter(pl.col('date_id').is_between(selected_dates[0], selected_dates[-1]))['weight'].to_numpy()
    
        # Train the model based on the type
        if model_name == 'lgb':
            # Train LightGBM model with early stopping and evaluation logging
            model.fit(X_tr, y_tr, w_tr, 
                        eval_metric=[r2_lgb],
                        eval_set = [(X_vl, y_vl, w_vl)],
                        callbacks = [
                            lgb.early_stopping(100),
                            lgb.log_evaluation(10)
                        ])
        elif model_name == 'xgb':
            # Train XGBoost model with early stopping and verbose logging
            model.fit(X_tr, y_tr, sample_weight=w_tr,
                        eval_set=[(X_vl, y_vl)],
                        sample_weight_eval_set=[w_vl],
                        verbose=10,
                        early_stopping_rounds=100)
    
        # Append the trained model to the list
        models.append(model)
    
        # Save the trained model to a file
        joblib.dump(model, f'{model_path}/{model_name}_{i}.model')
    
        # Delte training data to free up memory
        del X_tr
        del y_tr
        del w_tr
        gc.collect()

    else :
        models.append(joblib.load(f'{model_path}/{model_name}_{i}.model'))
    return    

# Custom R2 metric for XGBoost
def r2_xgb(y_true, y_pred, sample_weight):
    r2 = 1 - np.average((y_pred - y_true)**2, weights=sample_weight) / (np.average(y_true**2, weights=sample_weight) + 1e-38)
    return -r2

# Custom R2 metric for LightGBM
def r2_lgb(y_true, y_pred, sample_weight):
    r2 = 1 - np.average((y_pred - y_true)**2, weights=sample_weight) / (np.average(y_true**2, weights=sample_weight) + 1e-38)
    return 'r2', r2, True

# Dict to store different models and their configurations
model_dict = {
    'lgb': lgb.LGBMRegressor(n_estimators=500, device='gpu', gpu_use_dp=True, objective='l2'),
    'xgb': xgb.XGBRegressor(n_estimators=2000, learning_rate=0.1, max_depth=6, tree_method='hist', device='cuda', objective='reg:squarederror', eval_metric=r2_xgb, disable_default_eval_metric=True),
}

# Train models for each fold
for i in range(N_fold):
    #train(model_dict, 'lgb')
    train(model_dict, 'xgb')

In [ ]:
lags_ : pl.DataFrame | None = None

# Replace this function with your inference code.
# You can return either a Pandas or Polars dataframe, though Polars is recommended.
# Each batch of predictions (except the very first) must be returned within 10 minutes of the batch features being provided.
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # All the responders from the previous day are passed in at time_id == 0. We save them in a global variable for access at every time_id.
    # Use them as extra features, if you like.
    global lags_
    if lags is not None:
        lags_ = lags

    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6'),
    )
    
    feat = test[features].to_numpy()
    
    pred = [model.predict(feat) for model in models]
    pred = np.mean(pred, axis=0)
    
    predictions = predictions.with_columns(pl.Series('responder_6', pred.ravel()))

    # The predict function must return a DataFrame
    assert isinstance(predictions, pl.DataFrame | pd.DataFrame)
    # with columns 'row_id', 'responer_6'
    assert list(predictions.columns) == ['row_id', 'responder_6']
    # and as many rows as the test data.
    assert len(predictions) == len(test)

    return predictions



In [ ]:


inference_server = js_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            './input/test.parquet',
            './input/lags.parquet',
        )
    )

